In [ ]:
# Install the library
!pip -q install FireHR==0.1.2 pyhdf==0.10.2

In [ ]:
# Authenticate to use Google Earth Engine API
# requires a google developer account and a created project
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=L7Kof3eLgsfBAG43yktPvaHPTeq3QhxW6Ncx_7Td63w&tc=scCiS87T5AG23-a_JYJWnhm1B8zhgfjWk5Db_Vj4GZs&cc=c1HWp_aaRM5ssZpz-KxQZ_3485T-cRRoxDWyOLsU5fI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvv9GJ6BF6LkteRD-QnL05sZo3f-YgrHjZY6HQTr4hBSjIKS-gzt73w

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
from FireHR.data import RegionST, download_data
# Bounding box coordinates

#minx="-94.88571095638814" miny="30.14514620619975" maxx="-94.78928008215365" maxy="30.28536053650498" - Trinity River
#minx="-91.47505319919564" miny="29.50821478911770" maxx="-91.40178319567509" maxy="29.54368741117223 - Luisiana delta
# minx="-106.64373712579015" miny="35.84925461242930" maxx="-106.36866021281392" maxy="36.01168317037658 - River Basin
# minx="-123.62093391790084" miny="39.54403867886698" maxx="-123.40558673156958" maxy="39.59625307504254" - Laytonville
minx=-121.942154
miny=46.720492
maxx=-121.751341
maxy=46.847384

# minx=-123.62093391790084
# miny=39.54403867886698
# maxx=-123.40558673156958
# maxy=39.59625307504254


left = minx
right = maxx
bottom = miny
top = maxy
#################################################################

path_save   = Path('/content/drive/MyDrive/data/mt_rainier')
products    = ["USDA/NAIP/DOQQ"]  # Product id in google earth engine
bands       = ['R', 'G', 'B'] # Red, Green, Blue

R = RegionST(name         = 'Test', 
             bbox         = [left,bottom,right,top], 
             scale_meters = 1, 
             time_start   = '2015-03-01', 
             time_end     = '2016-07-25')

# Download time series
# download_data_ts(R, products, bands, path_save)

time_window = R.times[0], R.times[-1]

# Download median composite of the 3 least cloudy images within the time_window
download_data(R, time_window, products, bands, path_save, 
              show_progress=True)

#download_data_ts(R, products, bands, path_save, show_progress=True)



In [ ]:
import numpy as np
import cv2
from PIL import Image

def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(Image.fromarray(a))

In [ ]:
#The below code views the downloaded data
im = np.concatenate([open_tif(f'data/download.{b}.tif').read() for b in bands])
im = im.transpose(1,2,0).astype(np.float32)/1000
im *= (255.0/im.max())
im = im.astype(int)
print(im.shape)
imshow(im)